# Importing all the required libraries

In [220]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
from sklearn.impute import KNNImputer #for imputing the outlier values
filterwarnings('ignore')
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from sklearn.preprocessing import RobustScaler # for scaling 

In [241]:
#reading the raw data set into df called 'data'
data = pd.read_csv('data.csv')
#data.head(1).transpose()

In [242]:
#describing the data
#data.describe()

In [243]:
# we can use min max scaler to scale this, as most of the columns are ratios which values will lie in between 0 and 1
#col_needs_scaling

In [244]:
#making the copy of original data
data_copy = data.copy()
data_copy1 = data_copy.copy()

# Treatment of Outliers

In [245]:
# Custom function for calculating the number of outliers for each column 
#which are outside of standard IQR range (between 25th & 75th percentie)
def cal_num_outliers(data,col,q1,q3):
    Q1 = data[col].quantile(q1)
    Q3 = data[col].quantile(q3)
    IQR = Q3 - Q1
    data_outliers = data[(data[col] < Q1 - 1.5*IQR) | (data[col] > Q3 + 1.5*IQR)]
    return data_outliers.shape[0]
    

In [246]:
#Custom function for calculating the number of outliers for each column which are outside of q1 & q3 range
def cal_num_outliers1(data,col,q1,q3):
    Q_low = data[col].quantile(q1)
    Q_upper = data[col].quantile(q3)
    #IQR = Q3 - Q1
    data_outliers = data[(data[col] < Q_low) | (data[col] > Q_upper)][col]
    return data_outliers.shape[0]
    

In [247]:
#defining three empty lists(for calculating total number of outliers in each column for three Quartile ranges) 
#and appending the count of outliers retrieved from custom functions
lst_25_75 = []
lst_05_95 = []
lst_01_99 = []
for i in data.columns:
    lst_25_75.append(cal_num_outliers(data,i,0.25,0.75))
    lst_05_95.append(cal_num_outliers1(data,i,0.05,0.95))
    lst_01_99.append(cal_num_outliers1(data,i,0.01,0.99))

In [248]:
#defining a comparision data frame and placing the the total num of outlier values for each 'Feature'
df_comp = pd.DataFrame()
df_comp['No of Outliers for 0.25 & 0.75'] = lst_25_75
df_comp['No of Outliers for 0.05 & 0.95'] = lst_05_95
df_comp['No of Outliers for 0.01 & 0.99'] = lst_01_99
df_comp['Features'] = data.columns
df_comp.set_index('Features',inplace= True)

In [249]:
#displaying the comparision data frame
df_comp

,No of Outliers for 0.25 & 0.75,No of Outliers for 0.05 & 0.95,No of Outliers for 0.01 & 0.99
Features,,,
Bankrupt?,220,220,0
ROA(C) before interest and depreciation before interest,391,682,138
ROA(A) before interest and % after tax,561,680,138
ROA(B) before interest and depreciation after tax,432,682,138
Operating Gross Margin,320,682,138
Realized Sales Gross Margin,318,682,138
Operating Profit Rate,716,680,138
Pre-tax net Interest Rate,773,682,138
After-tax net Interest Rate,867,681,138


In [250]:
#to detect the outliers, we will get the index of all the outlier values and then remove these index from df
#finding all the outlier indexes in data and storing them in set to avoid duplicates(as same index can have multiple outliers)
#Method1
idx_set = set()
def remove_outliers(data,col,q1,q3):
    Q_low = data[col].quantile(q1)
    Q_upper = data[col].quantile(q3)
    #IQR = Q3 - Q1
    data_outliers = data[(data[col] < Q_low) | (data[col] > Q_upper)][col]
    idx = data_outliers.index
    idx_set.update(idx)
for i in data.columns:
    remove_outliers(data,i,0.01,0.99)
print('Total number of index values to be removed:',len(idx_set))
print('Total no of rows in the data after removing the outliers:',data.shape[0] - len(idx_set))

Total number of index values to be removed: 3227
Total no of rows in the data after removing the outliers: 3592


In [251]:
#Method 2:
#Replacing outliers with NaN values
for col in data_copy1.columns:
    Q_low = data_copy1[col].quantile(0.01)
    Q_upper = data_copy1[col].quantile(0.99)
    data_outliers = data_copy1[(data_copy1[col] < Q_low) | (data_copy1[col] > Q_upper)][col]
    for idx in data_outliers.index:
        data_copy1.loc[idx,col] = np.NaN

In [252]:
#dropping the NaN values results in 3592 rows (which are left after removing the outliers)
print('Total no of rows in the data after removing the outliers using Method2:',data_copy1.dropna().shape[0])

Total no of rows in the data after removing the outliers using Method2: 3592


In [253]:
#Method 3:
#print(data.quantile(0.01))
#print(data.quantile(0.99))
#print(data.shape[0])
print('Total no of rows in the data after removing the outliers using Method3:',
      data[~((data < data.quantile(0.01)) | (data > data.quantile(0.99)))].dropna().shape[0])

Total no of rows in the data after removing the outliers using Method3: 3592


In [254]:
#removing the outliers
#while removing the outliers missing values get introduced
#so these missing values are actually the outlier values

data_without_outliers = data[~((data < data.quantile(0.01)) | (data > data.quantile(0.99)))]

In [255]:
#we are considering to drop the row(index), if it has 11 outier values (here it is missing values) 
#we can use thresh parameter in drop na
#thresh states, min number of values in the row without missing values to be considered not to drop the row

data_thresh_10 = data_without_outliers.dropna(thresh = data_without_outliers.shape[1] - 10)
data_thresh_10.shape

(6525, 96)

In [256]:
imputer = KNNImputer()
x = imputer.fit_transform(data_thresh_10)
x.shape

(6525, 96)

In [257]:
data_cleaned = pd.DataFrame(data = x,columns=data_thresh_10.columns)

In [258]:
data_cleaned.shape

(6525, 96)

In [259]:
data_cleaned.isna().sum()

Bankrupt?                                                   0
 ROA(C) before interest and depreciation before interest    0
 ROA(A) before interest and % after tax                     0
 ROA(B) before interest and depreciation after tax          0
 Operating Gross Margin                                     0
 Realized Sales Gross Margin                                0
 Operating Profit Rate                                      0
 Pre-tax net Interest Rate                                  0
 After-tax net Interest Rate                                0
 Non-industry income and expenditure/revenue                0
 Continuous interest rate (after tax)                       0
 Operating Expense Rate                                     0
 Research and development expense rate                      0
 Cash flow rate                                             0
 Interest-bearing debt interest rate                        0
 Tax rate (A)                                               0
 Net Val

In [260]:
#taking a different data (small data set ) from seaborn
#df_small1 = sns.load_dataset('tips')
#df_small1.head()
#df_small1.shape[0]
#df_small = df_small1.select_dtypes(np.number)
#considering all the values which are greater than 1% and less than 99%
#df_small[~((df_small < df_small.quantile(0.01)) | (df_small > df_small.quantile(0.99)))].isna().sum()
#we have considered the range of values in between 1 and 99 percentile

#python compares all the values with this range, if the condition is satisfied it will pick the value else it will consider
#as missing value

#so this is why missing values are introduced in the data set

#lets drop NAN values and see the shape
#we have 5 outliers in total_bill and 3 outliers in tip 
#so in actual 8 values needs to be dropped

#df_small[~((df_small < df_small.quantile(0.01)) | (df_small > df_small.quantile(0.99)))].dropna().shape[0]

#calculating the difference between len of original data and len of data after outliers are dropped
#df_small1.shape[0] - df_small[~((df_small < df_small.quantile(0.01)) | (df_small > df_small.quantile(0.99)))].dropna().shape[0]

#so we had 8 outliers, so it should be 244 - 8 which is 236 (theory)
#but there is chance of outliers present in the same index(row) for the different columns
#which is why the actual len of data set with outliers removed is 238

#Instead of dropping these Outliers we can impute them using mean or trimmed mean

#data_fill_na = data_copy1.copy()
#for col in data_copy1.columns:
    #data_fill_na[col] = data_copy1[col].fillna(data_copy1[col].mean())

# Row wise outliers checking

In [261]:
#data_rows = data.transpose()

In [262]:
#lst3 = []
#lst4 = []
#lst5 = []
#for i in data_rows.columns:
    #lst.append(cal_num_outliers(data_rows,i,0.25,0.75))
    #lst1.append(cal_num_outliers(data_rows,i,0.05,0.95))
    #lst2.append(cal_num_outliers(data_rows,i,0.01,0.99))

In [263]:
#df_comp_row = pd.DataFrame()
#df_comp_row['No of Outliers for 0.25 & 0.75'] = lst3
#df_comp_row['No of Outliers for 0.05 & 0.95'] = lst4
#df_comp_row['No of Outliers for 0.01 & 0.99'] = lst5

In [264]:
#df_comp_row

In [265]:
#data.info()

In [266]:
#((data_rows < Q1 - 1.5*IQR) | (data_rows > Q3 + 1.5*IQR)).sum()

# Checking for Derived Variables

# Checking Correlation

In [267]:
#checking for co-relation
df_corr = data_cleaned.corr()
df_corr

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),Operating Expense Rate,Research and development expense rate,Cash flow rate,Interest-bearing debt interest rate,Tax rate (A),Net Value Per Share (B),Net Value Per Share (A),Net Value Per Share (C),Persistent EPS in the Last Four Seasons,Cash Flow Per Share,Revenue Per Share (Yuan ¥),Operating Profit Per Share (Yuan ¥),Per Share Net profit before tax (Yuan ¥),Realized Sales Gross Profit Growth Rate,Operating Profit Growth Rate,After-tax Net Profit Growth Rate,Regular Net Profit Growth Rate,Continuous Net Profit Growth Rate,Total Asset Growth Rate,Net Value Growth Rate,Total Asset Return Growth Rate Ratio,Cash Reinvestment %,Current Ratio,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Debt ratio %,Net worth/Assets,Long-term fund suitability ratio (A),Borrowing dependency,Contingent liabilities/Net worth,Operating profit/Paid-in capital,Net profit before tax/Paid-in capital,Inventory and accounts receivable/Net value,Total Asset Turnover,Accounts Receivable Turnover,Average Collection Days,Inventory Turnover Rate (times),Fixed Assets Turnover Frequency,Net Worth Turnover Rate (times),Revenue per person,Operating profit per person,Allocation rate per person,Working Capital to Total Assets,Quick Assets/Total Assets,Current Assets/Total Assets,Cash/Total Assets,Quick Assets/Current Liability,Cash/Current Liability,Current Liability to Assets,Operating Funds to Liability,Inventory/Working Capital,Inventory/Current Liability,Current Liabilities/Liability,Working Capital/Equity,Current Liabilities/Equity,Long-term Liability to Current Assets,Retained Earnings to Total Assets,Total income/Total expense,Total expense/Assets,Current Asset Turnover Rate,Quick Asset Turnover Rate,Working capitcal Turnover Rate,Cash Turnover Rate,Cash Flow to Sales,Fixed Assets to Assets,Current Liability to Liability,Current Liability to Equity,Equity to Long-term Liability,Cash Flow to Total Assets,Cash Flow to Liability,CFO to Assets,Cash Flow to Equity,Current Liability to Current Assets,Liability-Assets Flag,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
Bankrupt?,1.000000,-0.191894,-0.198699,-0.199960,-0.110607,-0.110101,-0.130956,-0.201959,-0.206882,-0.177211,-0.224278,-0.001178,-0.017912,-0.102205,-0.013611,-0.103269,-0.144970,-0.148789,-0.147658,-0.194170,-0.095704,-0.053211,-0.133837,-0.188473,-0.029441,-0.055531,-0.109973,-0.099670,-0.092974,-0.030133,-0.116448,-0.067768,-0.073654,-0.111728,-0.116272,-0.061269,0.259166,0.216862,-0.216862,-0.031194,0.286711,0.024351,-0.133089,-0.186880,0.093571,-0.073298,-0.018873,0.039495,-0.001739,0.086309,0.008940,0.012475,-0.103896,0.118814,-0.156043,-0.073977,-0.045282,-0.086894,-0.108815,-0.067505,0.164549,-0.097906,-0.009945,-0.008811,-0.028741,-0.150304,0.222638,-0.009534,-0.209001,-0.167911,0.053873,0.010076,0.034275,-0.089764,-0.019765,-0.056381,0.066988,-0.028741,0.222638,0.143770,-0.056071,-0.038188,-0.108484,-0.068241,0.211545,NaN,-0.223674,0.048055,-0.072201,-0.110606,-0.288289,0.253555,0.017764,-0.068787,NaN,-0.121866
ROA(C) before interest and depreciation before interest,-0.191894,1.000000,0.905785,0.963984,0.440476,0.438122,0.694309,0.725664,0.704194,0.373224,0.726400,0.048136,0.105785,0.430008,0.029166,0.316477,0.503003,0.504581,0.504270,0.825221,0.455531,0.227277,0.753031,0.808776,0.173895,0.278346,0.346772,0.345174,0.351351,0.016902,0.430133,0.466289,0.311000,0.164266,0.207429,0.015578,-0.249350,-0.2391

In [268]:
data_cleaned['Bankrupt?'].corr(data_cleaned[' ROA(C) before interest and depreciation before interest'])

-0.1918940864675842

In [269]:
#finding the names of highly correlated columns using custom function correlation

col_set = set()#filtered columns
def correlation(data_set,threshold):
    corr_matrix = data_set.corr()
    for i in range(len(corr_matrix)):
        for j in range(i):
            if corr_matrix.iloc[i,j] > threshold:
                col_set.add(corr_matrix.columns[i])
correlation(data_cleaned,0.8)

In [270]:
print('total number of col which are highly correlated are:',len(col_set))

total number of col which are highly correlated are: 34


In [271]:
data_no_correlation = data_cleaned.drop(col_set, axis = 1)

In [272]:
data_no_correlation.shape

(6525, 62)

In [273]:
data_no_correlation.var() == 0

Bankrupt?                                                   False
 ROA(C) before interest and depreciation before interest    False
 Operating Gross Margin                                     False
 Operating Profit Rate                                      False
 Pre-tax net Interest Rate                                  False
 Non-industry income and expenditure/revenue                False
 Operating Expense Rate                                     False
 Research and development expense rate                      False
 Cash flow rate                                             False
 Interest-bearing debt interest rate                        False
 Tax rate (A)                                               False
 Net Value Per Share (B)                                    False
 Cash Flow Per Share                                        False
 Revenue Per Share (Yuan ¥)                                 False
 Realized Sales Gross Profit Growth Rate                    False
 Operating

In [274]:
#we can remove net income flag liability assets flag as var is 0

In [275]:
data_no_correlation.columns

Index(['Bankrupt?', ' ROA(C) before interest and depreciation before interest',
       ' Operating Gross Margin', ' Operating Profit Rate',
       ' Pre-tax net Interest Rate',
       ' Non-industry income and expenditure/revenue',
       ' Operating Expense Rate', ' Research and development expense rate',
       ' Cash flow rate', ' Interest-bearing debt interest rate',
       ' Tax rate (A)', ' Net Value Per Share (B)', ' Cash Flow Per Share',
       ' Revenue Per Share (Yuan ¥)',
       ' Realized Sales Gross Profit Growth Rate',
       ' Operating Profit Growth Rate', ' After-tax Net Profit Growth Rate',
       ' Continuous Net Profit Growth Rate', ' Total Asset Growth Rate',
       ' Net Value Growth Rate', ' Total Asset Return Growth Rate Ratio',
       ' Cash Reinvestment %', ' Current Ratio', ' Interest Expense Ratio',
       ' Total debt/Total net worth', ' Net worth/Assets',
       ' Long-term fund suitability ratio (A)',
       ' Contingent liabilities/Net worth',
       ' I

In [276]:
data_no_correlation.drop([' Net Income Flag',' Liability-Assets Flag'],axis = 1,inplace = True)

# Multicollinearity using VIF

In [277]:
#checking for multicollinearity with VIF
vif = pd.DataFrame()
vif_list = []
for idx in range(data_no_correlation.shape[1]):
    vif_list.append(VIF(data_no_correlation.values,idx))
vif['features'] = data_no_correlation.columns
vif['VIF'] = vif_list

In [278]:
columns_with_vif_greater_than_5 = vif[vif['VIF'] > 5]['features'].values

In [279]:
data_cleaned_vif = data_no_correlation.drop(columns_with_vif_greater_than_5,axis = 1)

In [280]:
data_cleaned_vif.shape

(6525, 52)

# Scaling

In [281]:
data_scaled = data_cleaned_vif.copy()

In [282]:
#finding all the columns which needs scaling based on the column max value
col_needs_scaling = []
for i in data_scaled.columns:
    maximum = data[i].max()
    if maximum > 1:
        col_needs_scaling.append(i)
        

In [291]:
len(col_needs_scaling)

20

In [292]:
data_cleaned_vif.shape[1]

52

In [284]:
RS = RobustScaler()
data_req_scaling = data_scaled[col_needs_scaling]
scaled = RS.fit_transform(data_req_scaling)

In [285]:
data_scaled_columns = pd.DataFrame(data=scaled,columns=col_needs_scaling)

In [286]:
data_scaled_columns.shape

(6525, 20)

In [287]:
data_scaled[col_needs_scaling] = data_scaled_columns

In [290]:
#data_cleaned.describe()
data_scaled.describe()

,Bankrupt?,ROA(C) before interest and depreciation before interest,Operating Gross Margin,Operating Profit Rate,Non-industry income and expenditure/revenue,Operating Expense Rate,Research and development expense rate,Cash flow rate,Interest-bearing debt interest rate,Tax rate (A),Net Value Per Share (B),Cash Flow Per Share,Revenue Per Share (Yuan ¥),Realized Sales Gross Profit Growth Rate,Operating Profit Growth Rate,After-tax Net Profit Growth Rate,Continuous Net Profit Growth Rate,Total Asset Growth Rate,Net Value Growth Rate,Total Asset Return Growth Rate Ratio,Cash Reinvestment %,Current Ratio,Interest Expense Ratio,Long-term fund suitability ratio (A),Contingent liabilities/Net worth,Accounts Receivable Turnover,Average Collection Days,Inventory Turnover Rate (times),Fixed Assets Turnover Frequency,Revenue per person,Operating profit per person,Allocation rate per person,Cash/Total Assets,Inventory/Working Capital,Inventory/Current Liability,Current Liabilities/Liability,Long-term Liability to Current Assets,Retained Earnings to Total Assets,Total income/Total expense,Total expense/Assets,Current Asset Turnover Rate,Quick Asset Turnover Rate,Working capitcal Turnover Rate,Cash Turnover Rate,Cash Flow to Sales,Fixed Assets to Assets,Equity to Long-term Liability,Cash Flow to Total Assets,Total assets to GNP price,No-credit Interval,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT)
count,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6.525000e+03,6525.000000,6525.000000,6.525000e+03,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6.525000e+03,6.525000e+03,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6.525000e+03,6525.000000,6.525000e+03,6525.000000,6525.000000,6525.000000,6.525000e+03,6.525000e+03,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000,6525.000000
mean,0.026054,0.506798,0.607982,0.999022,0.303525,4.855860e-01,0.387940,0.467151,2.469569e+10,0.110733,0.189583,0.323500,0.311256,0.022141,0.848082,0.689430,0.217599,-0.365102,0.399937,0.264125,0.380072,0.369678,0.630963,0.007046,0.005724,0.622029,0.077815,4.649485e-01,3.029708e+11,0.536691,0.399241,0.501903,0.119301,0.277326,2.482564e+08,0.766212,3.003870e+08,0.936473,0.002384,0.027653,4.363279e+12,4.481514e-01,0.593977,0.279129,0.671577,0.168498,0.114859,0.650092,0.991549,0.623849,0.026972,0.565493
std,0.159307,0.049518,0.010431,0.000150,0.000206,7.770086e-01,0.727781,0.009270,2.015020e+11,0.119523,0.025011,0.010471,1.066779,0.000208,0.000322,0.001002,0.000108,1.223598,1.577865,0.000693,0.012267,1.175341,0.001198,0.004367,0.000712,2.530709,0.776496,6.928698e-01,7.614526e+11,1.525841,0.014318,1.632466,0.122065,0.000763,4.608578e+09,0.196077,5.153356e+09,0.014698,0.000253,0.018736,1.027226e+13,6.933157e-01,0.000073,0.644469,0.000035,0.681266,0.005453,0.036671,3.441242,0.001190,0.000771,0.001409
min,0.000000,0.338956,0.581213,0.997638,0.302200,-4.258946e-14,-0.173178,0.437687,-1.006270e+00,0.000000,0.140618,0.282061,-0.871601,0.021676,0.846406,0.678531,0.216619,-2.832599,-2.428825,0.261932,0.321253,-0.983270,0.624254,0.004911,0.005366,-0.881020,-1.402252,-1.499445e-13,-1.570862e-01,-0.667658,0.357387,-0.745152,0.002658,0.273272,-8.502951e-01,0.205910,-2.376767e-01,0.854337,0.001837,0.004296,-3.574058e-01,-2.486251e-14,0.593795,-0.276018,0.671379,-0.709290,0.110933,0.524336,-0.454693,0.615681,0.025333,0.555637
25%,0.000000,0.478487,0.600672,0.998973,0.303470,-2.906539e-14,-0.173178,0.461783,-3.573668e-01,0.000000,0.174287,0.318101,-0.381672,0.022066,0.847988,0.689279,0.217581,-0.581498,-0.359202,0.263782,0.375135,-0.351476,0.630612,0.005252,0.005366,-0.351275,-0.476606,-1.341258e-13,-1.134787e-01,-0.326159,0.392687,-0.347182,0.034479,0.277046,-4.250165e-01,0.633721,-2.376767e-01,0.931754

# Anomalies

In [ ]:
#Check Anomalies

# Check for Imbalance in Target Variable

In [ ]:
#check proportion

# Univariate Analysis

In [ ]:
#check univariate Analysis

# BiVariate Analysis

In [ ]:
#check Bi Variate Analysis